In [68]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wandb
import json
import re
import ast
import litellm

In [82]:
df = pd.read_parquet("dbscan_hierarchical_mcs_50-2_with_embeddings.parquet")
df["model"] = df["model"].replace("_responses", "")
df.to_json("/home/lisabdunlap/model_cards/public/dbscan_hierarchical_mcs_50-2_with_embeddings.jsonl", orient="records", lines=True)

In [73]:
df.model.value_counts()

model
meta-llama_Meta-Llama-3-8B-Instruct         9340
Qwen_Qwen2.5-Coder-7B-Instruct              8716
meta-llama_Llama-3.1-8B-Instruct            8374
Qwen_Qwen3-8B                               6410
Qwen_Qwen2.5-Math-7B-Instruct               6073
deepseek-ai_DeepSeek-R1-Distill-Llama-8B    4966
OpenGVLab_InternVL3-8B                      4637
mistralai_Mistral-7B-Instruct-v0.3          4387
microsoft_phi-4                             3644
microsoft_phi-4_responses                   1228
Name: count, dtype: int64

In [65]:
pd.read_csv("/home/lisabdunlap/model_cards/public/all_one_sided_comparisons_clustered_4.csv").columns

/tmp/ipykernel_306544/335233499.py:1: DtypeWarning: Columns (17,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("/home/lisabdunlap/model_cards/public/all_one_sided_comparisons_clustered_4.csv").columns


Index(['prompt', 'model_1_response', 'model_2_response', 'model_1_name',
       'model_2_name', 'differences', 'parsed_differences', 'parse_error',
       'model', 'property_description', 'category', 'evidence', 'type',
       'reason', 'impact', 'unexpected_behavior', 'response',
       'response_property', 'message', 'e.g.', 'economics',
       'property_description_coarse_cluster_label',
       'property_description_fine_cluster_label',
       'property_description_coarse_cluster_id',
       'property_description_fine_cluster_id'],
      dtype='object')

In [48]:
df = pd.read_parquet("new_prompt_4o_big_with_embeddings.parquet")
cluster_names = df['property_description_cluster_label'].unique()

# sample 3 property_description for each cluster_name
fine_grained_properties_sampled = df.groupby('property_description_cluster_label').apply(lambda x: x.sample(10))['property_description'].tolist()
print(fine_grained_properties_sampled[:2])

prompt = """Below are a large list of peroperties seen in the responses of LLM A. I want to cluster these properties into at most 30 clusters. Please come up with these cluster names, where each cluster name is description (up to 10 words) of a LLM output property that accurately describes most or all of the properties in the cluster. Remeber each of these should be a property of a model response, not a category of properties. For instance "Speaking Tone and Emoji Usage" is a category of properties, but "uses an enthusiastic tone" or "uses emojis" is a property of a model response. Similarily, "various types of reasoning" is a category of properties, but "uses deductive reasoning to solve problems" or "uses inductive reasoning to solve problems" is a property of a model response. Think about whether a user could easily understand the models behavior at a detailed level by looking at the cluster name. Remeber to reflect on the clusters you made: are there any which are roughly equivalent? If so, you can combine them. Each cluster should provide a unique insight into the model's behavior that is not already provided by the other clusters.

Output your final cluster descriptions as a list with each cluster name on a new line.

Here are the values: {values_text}"""

from litellm import completion
cluster_str = "\n".join(cluster_names)
# print(len(fine_grained_properties_sampled))
cluster_str = "\n".join(fine_grained_properties_sampled)
response = completion(
    model="o4-mini-2025-04-16",
    messages=[{"role": "user", "content": prompt.format(values_text=cluster_str)}],
    caching=True,
    # max_tokens=4096
)
raw_output = response.choices[0].message.content.strip()
output = [x.strip() for x in raw_output.split("\n") if x.strip()]
output = ['.'.join(x.split(".")[1:]).strip() for x in output]
print(raw_output)

/tmp/ipykernel_306544/2073434424.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fine_grained_properties_sampled = df.groupby('property_description_cluster_label').apply(lambda x: x.sample(10))['property_description'].tolist()


['detailed, step-by-step reasoning and guidance on how to decide a political vote.', 'More concise and focused on actionable suggestions.']
1. Provides detailed step-by-step reasoning and guidance  
2. Gives concise, direct answers with minimal elaboration  
3. Offers action-oriented advice with clear, practical steps  
4. Delivers vague or generic suggestions lacking actionable detail  
5. Presents clear final recommendations or definitive conclusions  
6. Uses hedging or explicit uncertainty language  
7. States authoritative, unequivocal assertions without caveats  
8. Acknowledges a training or knowledge cutoff date  
9. Declines or defers answering due to policy, knowledge gaps, or sensitivity  
10. Requests clarification or additional information from the user  
11. Engages interactively by asking follow-up questions  
12. Adopts a formal, professional tone  
13. Uses a conversational, informal tone  
14. Employs an instructional or directive tone  
15. Conveys a supportive, enco

In [58]:
output = [x.strip() for x in raw_output.split("\n") if x.strip()]
output = ['.'.join(x.split(".")[1:]).strip() for x in output]
output

['Provides detailed step-by-step reasoning and guidance',
 'Gives concise, direct answers with minimal elaboration',
 'Offers action-oriented advice with clear, practical steps',
 'Delivers vague or generic suggestions lacking actionable detail',
 'Presents clear final recommendations or definitive conclusions',
 'Uses hedging or explicit uncertainty language',
 'States authoritative, unequivocal assertions without caveats',
 'Acknowledges a training or knowledge cutoff date',
 'Declines or defers answering due to policy, knowledge gaps, or sensitivity',
 'Requests clarification or additional information from the user',
 'Engages interactively by asking follow-up questions',
 'Adopts a formal, professional tone',
 'Uses a conversational, informal tone',
 'Employs an instructional or directive tone',
 'Conveys a supportive, encouraging, or empathetic tone',
 'Writes in a poetic, narrative, or creative style',
 'Incorporates emojis in the response',
 'Uses LaTeX formatting, mathematical 

In [59]:
from litellm import completion
import litellm
from sklearn.neighbors import NearestNeighbors

# embed each output
embeddings = []
for output in output:
  response = litellm.embedding(
    model="text-embedding-3-small",
    input=output,
    caching=True
  )
  embeddings.append(response['data'][0]['embedding'])

def get_embedding(x):
  response = litellm.embedding(
    model="text-embedding-3-small",
    input=x,
    caching=True
  )
  output = response['data'][0]['embedding']
  # normalize the embedding
  output = output / np.linalg.norm(output)
  return output

label_to_embedding = {}
for label in df["property_description_cluster_label"].unique():
  label_to_embedding[label] = get_embedding(label)

df["property_description_cluster_label_embedding"] = df['property_description_cluster_label'].apply(lambda x: label_to_embedding[x])

cluster_embeddings = np.array(embeddings)
#normalize the cluster embeddings
cluster_embeddings = cluster_embeddings / np.linalg.norm(cluster_embeddings, axis=1, keepdims=True)
print(cluster_embeddings)

[[-0.00066448  0.00604588  0.0249472  ... -0.01478713 -0.05720186
   0.0068807 ]
 [-0.02188785  0.00712499  0.02016898 ... -0.01088848  0.01843625
   0.03537541]
 [-0.00342984  0.02584502 -0.01316307 ... -0.01320818 -0.02991949
   0.03455024]
 ...
 [ 0.00760525 -0.00265432 -0.00024325 ...  0.01236299 -0.01314999
   0.02139199]
 [ 0.0110149  -0.01870955  0.01200267 ... -0.02798963 -0.03433129
   0.01857675]
 [-0.0364133   0.01100867  0.04894127 ... -0.0344955  -0.02747187
   0.00942711]]


In [60]:
# for each embedding in df_embedding, get the index of the neasrest neighbor in cluster_embeddings

nn = NearestNeighbors(n_neighbors=1, metric='cosine')
df_embeddings = np.stack(df["property_description_cluster_label_embedding"])
print(df_embeddings.shape)
nn.fit(cluster_embeddings) # Fit on the cluster embeddings (30 points)

# Get nearest neighbor index for each df embedding (57k points)
distances, indices = nn.kneighbors(df_embeddings)
df["nn"] = indices.flatten() # This will give values 0-29 for each row



(57847, 1536)


In [61]:
df['nn'] = df['nn'].apply(lambda x: cluster_names[x])
df

,prompt,model_1_response,model_2_response,model_1_name,model_2_name,differences,parsed_differences,parse_error,model,property_description,...,response,response_property,message,e.g.,economics,property_description_cluster_label,property_description_cluster_id,property_description_embedding,property_description_cluster_label_embedding,nn
0,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Data Handling', 'e.g.': None, 'e...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,Uses hypothetical data to illustrate the proce...,...,None,None,None,None,None,Speculative and exploratory reasoning.,105,"[-1.2275167429139555, -1.2883683669471107, 0.3...","[0.026008254016026803, 0.002357548344534629, 0...",Requests Clarification and Context
1,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Realism', 'e.g.': None, 'economi...",NaN,Qwen_Qwen3-8B,Provides real-world context and references spe...,...,None,None,None,None,None,Detailed and Data-Driven Responses,49,"[-1.881744495456676, -0.5725609749467968, 0.78...","[-0.035214136655136376, 0.031213238276193787, ...",Concise and minimal elaboration.
2,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Problem Solving Style', 'e.g.': ...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,"Follows a structured, mathematical approach wi...",...,None,None,None,None,None,Detailed step-by-step explanations and calcula...,100,"[-0.8315669518327166, -1.2080537895687322, 0.1...","[-0.008470963376647917, -0.00616070114555629, ...",Speculative and exploratory reasoning.
3,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Clarity', 'e.g.': None, 'economi...",NaN,Qwen_Qwen3-8B,Discusses real-world nuances and potential amb...,...,None,None,None,None,None,Emphasizes Uncertainty and Multiple Interpreta...,101,"[1.0838049158301652, 0.07711026204734685, 0.66...","[0.02982220621642065, 0.03305136216118123, 0.0...",Comprehensive code examples and detailed expla...
4,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Accuracy', 'e.g.': None, 'econom...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,Uses the term 'sieveups' instead of 'seizures'...,...,None,None,None,None,None,Corrupted and incoherent language generation.,75,"[-0.8818394915388433, -0.830549969555021, -0.4...","[-0.01881886990820409, 0.019115341544282658, 0...",Conceptual and Practical Emphasis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57842,how might i instrument firebase calls to see w...,\n\nTo address the high number of reads in you...,To instrument McGregor calls and see why you h...,deepseek-ai_DeepSeek-R1-Distill-Llama-8B,Qwen_Qwen2.5-Math-7B-Instruct,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Code Relevance', 'e.g.': None, '...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,Provides Python code example unrelated to Fire...,...,None,None,None,None,None,Technical specificity and accuracy.,41,"[-0.39392778816570845, -0.1564127270528338, -0...","[0.0119752811346808, 0.055114966779423805, 0.0..."

In [46]:
df['nn'] = df['nn'].apply(lambda x: cluster_names[x])
df.to_parquet("new_prompt_4o_big_with_embeddings_nn.parquet")

In [47]:
pd.read_parquet("new_prompt_4o_big_with_embeddings_nn.parquet")

,prompt,model_1_response,model_2_response,model_1_name,model_2_name,differences,parsed_differences,parse_error,model,property_description,...,unexpected_behavior,response,response_property,message,e.g.,economics,property_description_cluster_label,property_description_cluster_id,property_description_embedding,nn
0,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Data Handling', 'e.g.': None, 'e...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,Uses hypothetical data to illustrate the proce...,...,False,None,None,None,None,None,Speculative and exploratory reasoning.,105,"[-1.2275167429139555, -1.2883683669471107, 0.3...",Conceptual and Practical Emphasis
1,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Realism', 'e.g.': None, 'economi...",NaN,Qwen_Qwen3-8B,Provides real-world context and references spe...,...,False,None,None,None,None,None,Detailed and Data-Driven Responses,49,"[-1.881744495456676, -0.5725609749467968, 0.78...",Refusal and Avoidance
2,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Problem Solving Style', 'e.g.': ...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,"Follows a structured, mathematical approach wi...",...,False,None,None,None,None,None,Detailed step-by-step explanations and calcula...,100,"[-0.8315669518327166, -1.2080537895687322, 0.1...",Detailed and Data-Driven Responses
3,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Clarity', 'e.g.': None, 'economi...",NaN,Qwen_Qwen3-8B,Discusses real-world nuances and potential amb...,...,False,None,None,None,None,None,Emphasizes Uncertainty and Multiple Interpreta...,101,"[1.0838049158301652, 0.07711026204734685, 0.66...","Friendly, conversational tone with emojis."
4,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Accuracy', 'e.g.': None, 'econom...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,Uses the term 'sieveups' instead of 'seizures'...,...,True,None,None,None,None,None,Corrupted and incoherent language generation.,75,"[-0.8818394915388433, -0.830549969555021, -0.4...",Detailed mathematical explanations and calcula...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57842,how might i instrument firebase calls to see w...,\n\nTo address the high number of reads in you...,To instrument McGregor calls and see why you h...,deepseek-ai_DeepSeek-R1-Distill-Llama-8B,Qwen_Qwen2.5-Math-7B-Instruct,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'category': 'Code Relevance', 'e.g.': None, '...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,Provides Python code example unrelated to Fire...,...,True,None,None,None,None,None,Technical specificity and accuracy.,41,"[-0.39392778816570845, -0.1564127270528338, -0...",Structured and comprehensive response format.
57843,how might i instrument firebase calls to see w...,\n\nTo address the high number of reads in you...,To instrument McGregor calls and see why you h...,deepseek-ai_DeepSeek-R1-Distill-Llama-8B,Qwen_Qwen2.5-Math-7B-Instruct,"```json\n[\n {\n ""model"": ""Qwe

## Other Stuff

In [4]:
from hierarchical_clustering import hierarchical_cluster_categories, load_clustered_results

df = load_clustered_results("differences/all_one_sided_comparisons_clustered_with_embeddings_oai.parquet")
df["property_description_coarse_cluster_label"] = (df["property_description_coarse_cluster_label"]
    .str.strip()  # Remove leading/trailing whitespace
    .str.replace(r'^["\']+|["\']+$', '', regex=True)  # Remove quotes only at start/end
    .str.replace(r'\s+', ' ', regex=True))  # Replace multiple spaces with single space
df["property_description_fine_cluster_label"] = (df["property_description_fine_cluster_label"]
    .str.strip()  # Remove leading/trailing whitespace 
    .str.replace(r'^["\']+|["\']+$', '', regex=True)  # Remove quotes only at start/end
    .str.replace(r'\s+', ' ', regex=True))  # Replace multiple spaces with single space
df["property_description"] = (df["property_description"]
    .str.strip()  # Remove leading/trailing whitespace
    .str.replace(r'^["\']+|["\']+$', '', regex=True)  # Remove quotes only at start/end
    .str.replace(r'\s+', ' ', regex=True))  # Replace multiple spaces with single space

# remove embeddings and save to csv
df["property_description_embedding"] = df["property_description_embedding"].apply(lambda x: json.dumps(x.tolist()))
# sabe parquet
df[['prompt', 'model_1_response', 'model_2_response', 'model_1_name',
       'model_2_name','model', 'property_description', 'category', 'evidence', 'type',
       'reason', 'impact', 'unexpected_behavior', 
       'property_description_coarse_cluster_label',
       'property_description_fine_cluster_label',
       'property_description_coarse_cluster_id',
       'property_description_fine_cluster_id', 'property_description_embedding']].to_parquet("/home/lisabdunlap/model_cards/public/all_one_sided_comparisons_clustered_with_embeddings_oai.parquet")
df_no_embeddings = df.drop(columns=["property_description_embedding"])
df_no_embeddings[['prompt', 'model_1_response', 'model_2_response', 'model_1_name',
       'model_2_name','model', 'property_description', 'category', 'evidence', 'type',
       'reason', 'impact', 'unexpected_behavior', 
       'property_description_coarse_cluster_label',
       'property_description_fine_cluster_label',
       'property_description_coarse_cluster_id',
       'property_description_fine_cluster_id']].to_csv("/home/lisabdunlap/model_cards/public/all_one_sided_comparisons_clustered_4_stripped.csv", index=False)

# convert embeddings to srting with embedding_str = json.dumps(embedding.tolist())
df["property_description_embedding"] = df["property_description_embedding"].apply(lambda x: json.dumps(x.tolist()))
df.to_csv("/home/lisabdunlap/model_cards/public/all_one_sided_comparisons_clustered_with_embeddings-clean.csv", index=False)

Loaded 57847 rows from differences/all_one_sided_comparisons_clustered_with_embeddings_oai.parquet
Available cluster columns: ['property_description_coarse_cluster_label', 'property_description_fine_cluster_label', 'property_description_coarse_cluster_id', 'property_description_fine_cluster_id']
Available embedding columns: ['property_description_embedding']
  property_description_embedding: dimension 1536


AttributeError: 'str' object has no attribute 'tolist'

In [ ]:
df.iloc[0]['property_description_embedding'].shape

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
df.iloc[0]['property_description_embedding'].shape

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
df.iloc[0]['property_description_embedding'].shape

AttributeError: 'str' object has no attribute 'shape'

In [13]:
df.iloc[0]['property_description_embedding'].shape

AttributeError: 'str' object has no attribute 'shape'

In [3]:
df.sample(100).to_csv("/home/lisabdunlap/model_cards/public/embedding_sample.csv", index=False)

In [4]:
pd.read_json("disguising/misc/arena_model_comparison_differences.jsonl", lines=True)
pd.read_csv("disguising/misc/arena_model_comparison_differences.csv")

,question_id,user_prompt,model_a_name,model_b_name,model_a_response,model_b_response,winner,differences,parsed_differences,parse_error
0,76ce56f8ba474768bc66128c7993ccb8,"php, handle tab in text as html, keeping them ...",mistral-large-2407,athene-70b-0725,Handling tabs in text as HTML and preserving t...,Here's a concise approach to handle tabs (`\t`...,model_b,"```json\n[\n {\n ""model"": ""athene-70b-0725...","[{'model': 'athene-70b-0725', 'property_descri...",NaN
1,e8fe7c9f75ab4e528367cc7de625c475,Is there any Artificial Superintelligence?,gemma-2-9b-it,qwen2-72b-instruct,"As a large language model, I am not able to pr...","As of now, true Artificial Superintelligence (...",model_b,"```json\n[\n {\n ""model"": ""gemma-2-9b-it"",...","[{'model': 'gemma-2-9b-it', 'property_descript...",NaN
2,6ccd7a51825249d5881ee501e06bb9ab,solve this sudoku:\n. 2 . | 6 . . | . . .\n. ....,mixtral-8x22b-instruct-v0.1,gemma-2-2b-it,Here is the completed Sudoku puzzle:\n\n8 2 3 ...,I can't solve Sudoku puzzles visually. I'm a ...,model_a,"```json\n[\n {\n ""model"": ""mixtral-8x22b-i...","[{'model': 'mixtral-8x22b-instruct-v0.1', 'pro...",NaN
3,463aa4efacf34f27b6a5c3f1f7417e86,paraphrase and simplify as best you can: The s...,gemini-1.5-pro-api-0514,reka-flash-preview-20240611,The most crucial thing to consider when judgin...,The key choice when assessing a business is it...,model_a,"```json\n[\n {\n ""model"": ""gemini-1.5-pro-...","[{'model': 'gemini-1.5-pro-api-0514', 'propert...",NaN
4,0d1a90cbff6d437f9f6a10c6c1e62437,"Which flag has blue in the center, red at the ...",llama-3.1-8b-instruct,gpt-3.5-turbo-0125,**Flag Identification**\n\nBased on the descri...,The flag you are describing is the flag of Rus...,model_b,"```json\n[\n {\n ""model"": ""llama-3.1-8b-in...","[{'model': 'llama-3.1-8b-instruct', 'property_...",NaN
...,...,...,...,...,...,...,...,...,...,...
1685,1f9303f90ecd46ecb823cf5fd634caa5,정수 배열 numbers가 매개변수로 주어집니다. numbers의 원소의 평균값을 ...,claude-3-5-sonnet-20240620,claude-3-haiku-20240307,제공된 코드는 정수 배열 `numbers`의 평균값을 계산하는 `solution` ...,코드가 잘 작성되었습니다. 정수 배열 `numbers`의 원소들의 평균값을 계산하여...,model_b,"```json\n[\n {\n ""model"": ""claude-3-5-sonn...",NaN,Unterminated string starting at: line 6 column...
1686,22bec5d88f7f47918c4b6d0e621c6852,My father died today.,command-r,gemma-2-9b-it,I'm so sorry for your loss. Losing a parent is...,I'm so sorry to hear that. Losing a father is ...,model_a,"```json\n[\n {\n ""model"": ""command-r"",\n ...","[{'model': 'command-r', 'property_description'...",NaN
1687,12c5e53ea2a04b179d765f65b60fdd27,I'm thinking about a post-apocalyptic scenario...,deepseek-v2-api-0628,claude-3-5-sonnet-20240620,Certainly! In a post-apocalyptic scenario wher...,In a post-apocalyptic scenario where existing ...,model_b,"```json\n[\n {\n ""model"": ""deepseek-v2-api...","[{'model': 'deepseek-v2-api-0628', 'property_d...",NaN
1688,fef1bf595f9d4a3fa8f5b27f1a36e6fa,"Generate list of 50 variants Security breach, ...",llama-3.1-8b-instruct,athene-70b-0725,**Cybersecurity Alert Variants**\n\nHere are 5...,"Here are 50 variants of ""Security breach, syst...",model_a,"```json\n[\n {\n ""model"": ""athene-70b-0725...","[{'model': 'athene-70b-0725', 'property_descri...",NaN


In [7]:
df = pd.read_json("all_one_sided_comparisons_clustered_4.jsonl", lines=True)
# df.property_description_coarse_cluster_label.unique()
# remove any extra ", \n, ', or " from the label
# Clean up coarse cluster labels by removing special characters and extra whitespace
df["property_description_coarse_cluster_label"] = (df["property_description_coarse_cluster_label"]
    .str.strip()  # Remove leading/trailing whitespace
    .str.replace(r'^["\']+|["\']+$', '', regex=True)  # Remove quotes only at start/end
    .str.replace(r'\s+', ' ', regex=True))  # Replace multiple spaces with single space
df["property_description_fine_cluster_label"] = (df["property_description_fine_cluster_label"]
    .str.strip()  # Remove leading/trailing whitespace 
    .str.replace(r'^["\']+|["\']+$', '', regex=True)  # Remove quotes only at start/end
    .str.replace(r'\s+', ' ', regex=True))  # Replace multiple spaces with single space
df["property_description"] = (df["property_description"]
    .str.strip()  # Remove leading/trailing whitespace
    .str.replace(r'^["\']+|["\']+$', '', regex=True)  # Remove quotes only at start/end
    .str.replace(r'\s+', ' ', regex=True))  # Replace multiple spaces with single space
df.to_csv("/home/lisabdunlap/model_cards/public/all_one_sided_comparisons_clustered_4.csv", index=False)
df

,prompt,model_1_response,model_2_response,model_1_name,model_2_name,differences,parsed_differences,parse_error,model,property_description,...,unexpected_behavior,response,response_property,message,e.g.,economics,property_description_coarse_cluster_label,property_description_fine_cluster_label,property_description_coarse_cluster_id,property_description_fine_cluster_id
0,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'model': 'Qwen_Qwen2.5-Math-7B-Instruct', 'pr...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,Uses hypothetical data to illustrate the proce...,...,False,None,None,None,None,None,Variations in AI response techniques,Hypothetical and Speculative Reasoning in Resp...,2,24
1,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'model': 'Qwen_Qwen3-8B', 'property_descripti...",NaN,Qwen_Qwen3-8B,Provides real-world context and references spe...,...,False,None,None,None,None,None,Detailed and diverse AI content attributes,Detail-oriented Technical Analysis and Implica...,0,30
2,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'model': 'Qwen_Qwen2.5-Math-7B-Instruct', 'pr...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,"Follows a structured, mathematical approach wi...",...,False,None,None,None,None,None,Step-by-step structured problem-solving approach,"Structured, Step-by-Step Explanation Approach",15,156
3,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'model': 'Qwen_Qwen3-8B', 'property_descripti...",NaN,Qwen_Qwen3-8B,Discusses real-world nuances and potential amb...,...,False,None,None,None,None,None,Variations in AI response techniques,Hypothetical and Speculative Reasoning in Resp...,2,24
4,Can you help me to find the country with the m...,To find the country with the most rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Math-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'model': 'Qwen_Qwen2.5-Math-7B-Instruct', 'pr...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,Uses the term 'sieveups' instead of 'seizures'...,...,True,None,None,None,None,None,Evaluation of AI's Historical and Cultural Kno...,Assessments of AI Model's Responses,18,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57842,how might i instrument firebase calls to see w...,\n\nTo address the high number of reads in you...,To instrument McGregor calls and see why you h...,deepseek-ai_DeepSeek-R1-Distill-Llama-8B,Qwen_Qwen2.5-Math-7B-Instruct,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'model': 'Qwen_Qwen2.5-Math-7B-Instruct', 'pr...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,Provides Python code example unrelated to Fire...,...,True,None,None,None,None,None,Detailed Technical Responses with Coding Examples,Firebase and caching in async computing,7,34
57843,how might i instrument firebase calls to see w...,\n\nTo address the high number of reads in you...,To instrument McGregor calls and see why you h...,deepseek-ai_DeepSeek-R1-Distill-Llama-8B,Qwen_Qwen2.5-Math-7B-Instruct,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Ma...","{'model': 'Qwen_Qwen2.5-Math-7B-Instruct', 'pr...",NaN,Qwen_Qwen2.5-Math-7B-Instruct,Includes Chinese text in the response.,...,False,None,None,None,None,None,Incoherent and

In [6]:
df.property_description_coarse_cluster_label.unique()

array(['Diverse Tone and Content in Responses',
       'AI Refusal to Engage with Prompts',
       'Detailed Music Analysis and Examples',
       'Incoherent and Nonsensical AI Responses',
       'Azure AD User Administration Techniques',
       'Elephant Trunk Anatomy Explanations',
       'SNS usage and best practices explanations',
       'Interpreting and Exploring Prompt Ambiguity',
       'AI Responses Regarding Anthem Modifications',
       'AI Refusal to Generate Lyrics/Poems',
       'Errors and Inaccuracies in AI Responses',
       'AI Admitting Lack of Knowledge',
       "Asmongold's identification and description errors/correctness",
       "Responses Interpreting 'Stonly guide' Term",
       'Types of AI Self-Introduction Styles',
       'Varied Tone and Structure AI Responses',
       'Handling of Final Probability in Responses',
       'Hypotheticals on 1+1=2 Being False Discussions',
       'Analysis of Religious Enlightenment Processes',
       'Incoherent or Irrelevan

In [2]:
files = os.listdir("disguising/misc/one_sided_comparisons")
total_df = []
for file in files:
  if ".jsonl" in file:
    df = pd.read_json(f"disguising/misc/one_sided_comparisons/{file}", lines=True).dropna(subset=['parsed_differences']).reset_index(drop=True)
    # df["parsed_differences"] = df["parsed_differences"].apply(ast.literal_eval)
    df = df.explode("parsed_differences")
    total_df.append(df)
total_df = pd.concat(total_df)

# count any remove any rows where parsed_differences is empty
total_df = total_df[total_df["parsed_differences"].notna()]

# append all the parsed differences dict to column values in df
behavior = pd.DataFrame(total_df["parsed_differences"].tolist())
# Drop any duplicate index values before concatenating
total_df = total_df.reset_index(drop=True)
behavior = behavior.reset_index(drop=True)
total_df = pd.concat([total_df, behavior], axis=1)

def remove_things(x):
  x = x[x.find('_')+1:]
  x = x.replace("-Instruct", "")
  return x.lower()

def model_name_pass(row):
  model_a = row["model_1_name"]
  model_b = row["model_2_name"]
  model_a_modified_name = remove_things(model_a)
  model_b_modified_name = remove_things(model_b)
  model = row["model"]
  if model == model_a or model.lower() == "model a" or remove_things(model) == model_a_modified_name:
    return model_a
  if model == model_b or model.lower() == "model b" or remove_things(model) == model_b_modified_name:
    return model_b
  else:
    # print(row["model"])
    # print(row["model_1_name"])
    # print(row["model_2_name"])
    # print("----")
    return None
total_df["model"] = total_df.apply(model_name_pass, axis=1)
len(total_df), len(total_df[total_df.model.isna()])


(57847, 72)

In [3]:
total_df.to_json("all_one_sided_comparisons.jsonl", lines=True, orient="records")

In [8]:
len(total_df.property_description.unique())

57042

In [25]:
# # disguising/misc/parsed_comparisons/32B--Qwen_Qwen2_deepseek-ai_DeepSeek-R1-Distill-Llama-8B.csv
# df = pd.read_json("disguising/misc/one_sided_comparisons/32B--Qwen_Qwen2.5-Coder-7B-Instruct_Qwen_Qwen3-8B.jsonl", lines=True).dropna(subset=['parsed_differences']).reset_index(drop=True)
# # df["parsed_differences"] = df["parsed_differences"].apply(ast.literal_eval)
# df = df.explode("parsed_differences")
# df.parsed_differences.iloc[0]
# # append all the parsed differences dict to column values in df
# behavior = pd.DataFrame(df["parsed_differences"].tolist())
# # Drop any duplicate index values before concatenating
# df = df.reset_index(drop=True)
# behavior = behavior.reset_index(drop=True)
# df = pd.concat([df, behavior], axis=1)
# df

In [3]:
from hierarchical_clustering import hierarchical_cluster_categories
df_clustered = hierarchical_cluster_categories(df, "property_description", context="properties seen in large language model responses", n_coarse_clusters=10, n_fine_clusters=200)
df_clustered

Computing embeddings for 2422 unique values using batch_api...
Embeddings computed in 27.87 seconds
Generating cluster summaries...
These are 50 representative values from a broad cluster of 957 total values, all properties seen in large language model responses: Incorrectly attributes Edward II's affair to Isabella of France, who was his wife., Provides a structured, explanatory breakdown of why some dinosaurs were large, including ecological and evolutionary reasons., Generalized list of climate-geology interactions with minimal examples., Declined to provide a specific starting lineup, citing lack of real-time data., Generic, repetitive descriptions for each piece (e.g., 'fast runs, trills, and a highly expressive melody')., Emphasized randomized compiling as a technique for error mitigation and noise averaging., Provides a concise and direct riddle with no elaboration., Correctly identifies C.J. 'Crazy' Cale, played by Kelly Ripa, as the toughest female lifeguard., Verifies the cal

,prompt,model_1_response,model_2_response,model_1_name,model_2_name,differences,parsed_differences,parse_error,model,property_description,category,evidence,type,reason,impact,unexpected_behavior,property_description_coarse_cluster_label,property_description_fine_cluster_label,property_description_coarse_cluster_id,property_description_fine_cluster_id
0,Can you help me to find the country with the m...,To find the country with the most Rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Coder-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Co...","{'model': 'Qwen_Qwen2.5-Coder-7B-Instruct', 'p...",None,Qwen_Qwen2.5-Coder-7B-Instruct,Hypothetical example used in place of real-tim...,Data Handling,Provides a hypothetical example with fictional...,General,This model typically avoids making factual cla...,High,False,AI language model response characteristics,Speculative and Uncertainty Indications in Res...,0,26
1,Can you help me to find the country with the m...,To find the country with the most Rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Coder-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Co...","{'model': 'Qwen_Qwen3-8B', 'property_descripti...",None,Qwen_Qwen3-8B,Direct factual claim about Kenya having the mo...,Fact Presentation,States: 'Kenya had the highest total weight of...,General,This model confidently presents a specific cou...,High,False,AI language model response characteristics,Inconsistent accuracy in factual information,0,0
2,Can you help me to find the country with the m...,To find the country with the most Rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Coder-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Co...","{'model': 'Qwen_Qwen2.5-Coder-7B-Instruct', 'p...",None,Qwen_Qwen2.5-Coder-7B-Instruct,"Structured, step-by-step explanation of method...",Explanation Style,"Lists five clear steps: data source, collectio...",General,This model tends to provide procedural clarity...,Medium,False,"""Structured, Detailed Explanations with Code E...","""Step-by-step structured explanations""",7,38
3,Can you help me to find the country with the m...,To find the country with the most Rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Coder-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Co...","{'model': 'Qwen_Qwen3-8B', 'property_descripti...",None,Qwen_Qwen3-8B,Inclusion of year-specific details and approxi...,Detail Level,"Mentions 'over 1,500 kg in 2021' and 'over 1,2...",Context-Specific,"This model provides granular, year-specific we...",High,False,AI language model response characteristics,Detailed Analysis in Large Language Models,0,5
4,Can you help me to find the country with the m...,To find the country with the most Rhino horn s...,"\n\nBetween 2021 and 2022, **Kenya** was repor...",Qwen_Qwen2.5-Coder-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Co...","{'model': 'Qwen_Qwen3-8B', 'property_descripti...",None,Qwen_Qwen3-8B,Mentions multiple organizations as potential s...,Credibility Sources,References CITES and TRAFFIC as sources for ac...,Context-Specific,This model explicitly suggests external source...,Medium,False,AI language model response characteristics,Inconsistent accuracy in factual information,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2419,how might i instrument firebase calls to see w...,There are several ways to instrument Firebase ...,\n\nTo instrument Firebase calls and analyze w...,Qwen_Qwen2.5-Coder-7B-Instruct,Qwen_Qwen3-8B,"```json\n[\n {\n ""model"": ""Qwen_Qwen2.5-Co...","{'model': 'Qwen_Qwen3-8B', 'property_descripti...",None,Qwen_Qwen3-8B,"Detailed, verbose explanation with code snippe...",Content Style,"For example, in the Realtime Database, you can...",General,Qwen_Qwen3-8B's response is more conversation

In [7]:
df_clustered = pd.read_json("disguising/misc/32B--Qwen_Qwen2.5-Coder-7B-Instruct_Qwen_Qwen3-8B-clustered.jsonl", lines=True)
df_clustered.model.unique()
model_options1 = "Qwen_Qwen2.5-Coder-7B-Instruct"
model_options2 = "Qwen_Qwen3-8B"
def ranme_model(x):
  if "coder" in x.lower():
    return "Qwen_Qwen2.5-Coder-7B-Instruct"
  else:
    return "Qwen_Qwen3-8B"
df_clustered.model = df_clustered.model.apply(ranme_model)
df_clustered.to_csv("/home/lisabdunlap/model_cards/public/qwen2_mistral_small_clustered.csv", index=False)

In [61]:
# df = pd.read_csv("disguising/misc/json_comparrisons/model_comparison_differences_json_32B_Qwen_Qwen2_mistralai_Mistral-Small-3.csv").dropna(subset=['parsed_differences']).reset_index(drop=True)
# df["parsed_differences"] = df["parsed_differences"].apply(ast.literal_eval)
# df = df.explode("parsed_differences")
# df.parsed_differences.iloc[0]
# # append all the parsed differences dict to column values in df
# behavior = pd.DataFrame(df["parsed_differences"].tolist())
# # Drop any duplicate index values before concatenating
# df = df.reset_index(drop=True)
# behavior = behavior.reset_index(drop=True)
# df = pd.concat([df, behavior], axis=1)

In [ ]:
# import litellm
# import numpy as np
# from sklearn.cluster import AgglomerativeClustering
# from sklearn.metrics import pairwise_distances

# # Get embeddings for each category
# categories = df.category.unique()
# embeddings = []
# for category in categories:
#     response = litellm.embedding(
#         model="text-embedding-3-small",
#         input=category,
#         caching=True
#     )
#     embeddings.append(response['data'][0]['embedding'])

# # Convert to numpy array
# embeddings = np.array(embeddings)

# # Hierarchical clustering with 2 levels
# n_coarse_clusters = 10   # Broad categories
# n_fine_clusters = 50    # More specific sub-categories

# # Coarse level clustering
# coarse_clustering = AgglomerativeClustering(
#     n_clusters=n_coarse_clusters, 
#     linkage='ward'
# )
# coarse_clusters = coarse_clustering.fit_predict(embeddings)

# # Fine level clustering  
# fine_clustering = AgglomerativeClustering(
#     n_clusters=n_fine_clusters, 
#     linkage='ward'
# )
# fine_clusters = fine_clustering.fit_predict(embeddings)

# # Create mappings
# category_to_coarse_cluster = dict(zip(categories, coarse_clusters))
# category_to_fine_cluster = dict(zip(categories, fine_clusters))

# # Group categories by coarse clusters
# coarse_cluster_categories = {}
# for category, cluster in category_to_coarse_cluster.items():
#     if cluster not in coarse_cluster_categories:
#         coarse_cluster_categories[cluster] = []
#     coarse_cluster_categories[cluster].append(category)

# # Group categories by fine clusters
# fine_cluster_categories = {}
# for category, cluster in category_to_fine_cluster.items():
#     if cluster not in fine_cluster_categories:
#         fine_cluster_categories[cluster] = []
#     fine_cluster_categories[cluster].append(category)

# # Get coarse cluster summaries
# coarse_cluster_summaries = {}
# for cluster, cats in coarse_cluster_categories.items():
#     prompt = f"These categories are grouped together in a broad cluster: {', '.join(cats)}. Please provide a short (1-3 words) high-level label that best describes what these categories have in common."
#     response = litellm.completion(
#         model="gpt-4",
#         messages=[{"role": "user", "content": prompt}],
#         caching=True
#     )
#     coarse_cluster_summaries[cluster] = response.choices[0].message.content.strip()

# # Get fine cluster summaries
# fine_cluster_summaries = {}
# for cluster, cats in fine_cluster_categories.items():
#     prompt = f"These categories are grouped together in a specific cluster: {', '.join(cats)}. Please provide a short (1-4 words) specific label that best describes what these categories have in common."
#     response = litellm.completion(
#         model="gpt-4",
#         messages=[{"role": "user", "content": prompt}],
#         caching=True
#     )
#     fine_cluster_summaries[cluster] = response.choices[0].message.content.strip()

# # Create mappings from category to cluster summaries
# category_to_coarse_summary = {
#     category: coarse_cluster_summaries[cluster]
#     for category, cluster in category_to_coarse_cluster.items()
# }

# category_to_fine_summary = {
#     category: fine_cluster_summaries[cluster]
#     for category, cluster in category_to_fine_cluster.items()
# }

# # Add both cluster levels as new columns in the dataframe
# df['coarse_cluster_label'] = df['category'].map(category_to_coarse_summary)
# df['fine_cluster_label'] = df['category'].map(category_to_fine_summary)
# df['coarse_cluster_id'] = df['category'].map(category_to_coarse_cluster)
# df['fine_cluster_id'] = df['category'].map(category_to_fine_cluster)

# # Display results
# print(f"Coarse clusters: {n_coarse_clusters}, Fine clusters: {n_fine_clusters}")
# print("\n" + "="*50)
# print("COARSE LEVEL CLUSTERS:")
# print("="*50)
# for cluster, summary in coarse_cluster_summaries.items():
#     print(f"\nCoarse Cluster {cluster} - {summary}:")
#     print(", ".join(coarse_cluster_categories[cluster]))

# print("\n" + "="*50)
# print("FINE LEVEL CLUSTERS:")
# print("="*50)
# for cluster, summary in fine_cluster_summaries.items():
#     print(f"\nFine Cluster {cluster} - {summary}:")
#     print(", ".join(fine_cluster_categories[cluster]))

# # Show hierarchical relationship
# print("\n" + "="*50)
# print("HIERARCHICAL RELATIONSHIP:")
# print("="*50)
# for coarse_cluster in range(n_coarse_clusters):
#     coarse_categories = set(coarse_cluster_categories[coarse_cluster])
#     print(f"\nCoarse Cluster {coarse_cluster} - {coarse_cluster_summaries[coarse_cluster]}:")
    
#     # Find which fine clusters belong to this coarse cluster
#     for fine_cluster in range(n_fine_clusters):
#         fine_categories = set(fine_cluster_categories[fine_cluster])
#         if fine_categories.issubset(coarse_categories):
#             print(f"  └── Fine Cluster {fine_cluster} - {fine_cluster_summaries[fine_cluster]}: {', '.join(fine_categories)}")

KeyboardInterrupt: 

In [72]:
df

,prompt,model_1_response,model_2_response,model_1_name,model_2_name,differences,parsed_differences,parse_error,difference,category,...,b_evidence,type,reason,impact,unexpected_behavior,cluster_label,coarse_cluster_label,fine_cluster_label,coarse_cluster_id,fine_cluster_id
0,Can you help me to find the country with the m...,Certainly! To determine the country with the m...,I can certainly help guide you on how to find ...,Qwen_Qwen2,mistralai_Mistral-Small-3,"```json\n[\n {\n ""difference"": ""Model A pr...",{'difference': 'Model A provided specific seiz...,NaN,Model A provided specific seizure data by coun...,Data Provision,...,I can suggest some steps and resources you can...,General,Model A is more likely to generate factual dat...,High,False,Guidance and Information Provision,Content Analysis,Technical Documentation Components,0,4
1,Can you help me to find the country with the m...,Certainly! To determine the country with the m...,I can certainly help guide you on how to find ...,Qwen_Qwen2,mistralai_Mistral-Small-3,"```json\n[\n {\n ""difference"": ""Model A pr...",{'difference': 'Model A listed estimated seizu...,NaN,Model A listed estimated seizure weights for e...,Specificity of Response,...,Look for academic papers or reports published ...,General,"Model A frequently includes detailed, albeit e...",High,False,Specificity and Clarity,Precision and Accuracy,"""Specificity Aspects""",1,24
2,Can you help me to find the country with the m...,Certainly! To determine the country with the m...,I can certainly help guide you on how to find ...,Qwen_Qwen2,mistralai_Mistral-Small-3,"```json\n[\n {\n ""difference"": ""Model A pr...",{'difference': 'Model A concluded with a clear...,NaN,"Model A concluded with a clear answer, while M...",Response Structure,...,Once you gather information from these sources...,General,"Model A tends to deliver direct answers, while...",Medium,False,Structural Organization Styles,Organizational Structures,Structural Organization,8,8
3,Hey can you write a sql script that includes b...,Certainly! Below is an example SQL script that...,Sure! Below is an example of an SQL script tha...,Qwen_Qwen2,mistralai_Mistral-Small-3,"```json\n[\n {\n ""difference"": ""Mode...",{'difference': 'Model A uses a `customer_id`-f...,NaN,Model A uses a `customer_id`-focused example w...,Example Structure,...,"product_id, SUM(sale_amount) > 1000 with WHERE...",Context-Specific,Both models responded to the request with `GRO...,Medium,False,Structural Organization Styles,Organizational Structures,Structural Organization,8,8
4,Hey can you write a sql script that includes b...,Certainly! Below is an example SQL script that...,Sure! Below is an example of an SQL script tha...,Qwen_Qwen2,mistralai_Mistral-Small-3,"```json\n[\n {\n ""difference"": ""Mode...",{'difference': 'Model A provides a more detail...,NaN,Model A provides a more detailed explanation o...,Explanation Style,...,Includes a 'Notes' section about `YEAR` functi...,General,Model B tends to provide technical footnotes a...,Medium,False,Structural Organization Styles,Content Analysis,Communication Styles,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,Does the following message have any grammar or...,"Sure, please provide the message you'd like me...","Of course, I'd be happy to help. Here's the me...",Qwen_Qwen2,mistralai_Mistral-Small-3,"```json\n[\n {\n ""difference"": ""Mode...",{'difference': 'Model A was concise and to the...,NaN,"Model A was concise and to the point, while Mo...",Response Style,...,"Of course, I'd be happy to help. Here's the me...",General,"Model A tends to be minimal and direct, wherea...",Medium,False,Narrative Analysis and Interpretation,Communication Analysis,Communication Evaluation Metrics,3,18
1485,Does the following message have any grammar or...,"Sure, please provide the message you'd like me...","Of course, I'd be happy to help. Here's the me...",Qwen_Qwen2,mistralai_Mistral-Small-3,"

In [73]:
df.to_csv("~/model_cards/public/qwen2_mistral_small.csv", index=False)

In [32]:
df[df['unexpected_behavior'] == 'True'].iloc[5]["parsed_differences"]


{'difference': 'Model A listed 5 characters with repetitive and incorrect names, while Model B provided accurate and distinct character names.',
 'category': 'Accuracy',
 'a_evidence': 'Vault-Tec-19 Vault-Tec-19 Vault-Tec-19... (repeats multiple times)',
 'b_evidence': 'Amara, the Siren; Moze, the Gunner; FL4K, the Beastmaster; Rhys, the Operator; Tannis',
 'type': 'Context-Specific',
 'reason': "Model A's response appears to be a breakdown or repetition triggered by the prompt, while Model B accurately lists the intended Borderlands 3 characters.",
 'impact': 'High',
 'unexpected_behavior': 'True'}